In [13]:
import re
import urllib.request
import zipfile
import nltk
nltk.download('punkt')
from lxml import etree
from nltk.tokenize import word_tokenize, sent_tokenize
from gensim.models import KeyedVectors
import csv
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [2]:
import pandas as pd
import csv
import sys

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [94]:
import os
rows = []
csv.field_size_limit(sys.maxsize)
for filename in os.listdir("/content/drive/MyDrive/IR-based change suggestion/dataset/issue data in csv"):
   with open(os.path.join("/content/drive/MyDrive/IR-based change suggestion/dataset/issue data in csv", filename), 'r') as f:

#file = open('/spark_issues.csv')
      type(f)
      csvreader = csv.reader(f)
      next(csvreader)
      for row in csvreader:
        i = 0
        for line in row:
          if i == 3:
            rows.append(line)
          i+=1

      f.close()
print(len(rows))

188909


In [64]:
def camel_case_split(identifier):
    return re.sub('([A-Z][a-z]+)', r' \1', re.sub('([A-Z]+)', r' \1',identifier)).split()

In [97]:
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
lemmatizer = WordNetLemmatizer()
i = 0
normalized_lines = []
for row in rows:
  content_text = re.sub(r'\([^)]*\)', '', row)
  lemmatized_text = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(content_text)]
  split_text = []
  for string in lemmatized_text:
     string_processing = camel_case_split(string)
     for word in string_processing:
         split_text.append(re.sub(r"[^a-z0-9]+", " ", word.lower()))
     normalized_lines.append(split_text)
for lists in normalized_lines:
  if ' ' in lists:
     lists.remove(' ')
  if 'a' in lists:
     lists.remove('a')
  if 'the' in lists:
     lists.remove('the')       
  
  
#for line in normalized_lines:141693
  #if i < 3:
#print(len(normalized_lines))
#print(normalized_lines)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [98]:
print(normalized_lines[:10])

[['reduce', 'multiple', 'file', 'attribute', 'call', 'of', 'java', 'utils', 'delete', 'recursively', 'using', 'java', 'io'], ['reduce', 'multiple', 'file', 'attribute', 'call', 'of', 'java', 'utils', 'delete', 'recursively', 'using', 'java', 'io'], ['reduce', 'multiple', 'file', 'attribute', 'call', 'of', 'java', 'utils', 'delete', 'recursively', 'using', 'java', 'io'], ['reduce', 'multiple', 'file', 'attribute', 'call', 'of', 'java', 'utils', 'delete', 'recursively', 'using', 'java', 'io'], ['reduce', 'multiple', 'file', 'attribute', 'call', 'of', 'java', 'utils', 'delete', 'recursively', 'using', 'java', 'io'], ['reduce', 'multiple', 'file', 'attribute', 'call', 'of', 'java', 'utils', 'delete', 'recursively', 'using', 'java', 'io'], ['reduce', 'multiple', 'file', 'attribute', 'call', 'of', 'java', 'utils', 'delete', 'recursively', 'using', 'java', 'io'], ['reduce', 'multiple', 'file', 'attribute', 'call', 'of', 'java', 'utils', 'delete', 'recursively', 'using', 'java', 'io'], ['reduc

In [99]:
from gensim.models import FastText

# embedding_size = 60
# window_size = 40
# min_word = 5
# down_sampling = 1e-2
# model = FastText(normalized_lines,
#                   size=embedding_size,
#                   window=window_size,
#                   min_count=min_word,
#                   sample=down_sampling,
#                   sg=1,
#                   iter=100)
model = FastText(normalized_lines, size=100, window=5, min_count=5, workers=4, sg=1)

#print(model.wv.vocab)

In [100]:
model.wv.most_similar("work")

[('do', 0.7015838623046875),
 ('properly', 0.7002865076065063),
 ('correctly', 0.6675848364830017),
 ('correctlly', 0.6659146547317505),
 ('n t', 0.6481700539588928),
 ('respect', 0.61865234375),
 ('support', 0.6088021993637085),
 ('workitem', 0.60353684425354),
 ('correctry', 0.5953583121299744),
 ('corrects', 0.5846196413040161)]

In [103]:
model.save("/content/drive/MyDrive/IR-based change suggestion/dataset/CGmodel.bin")

In [105]:
print(model)

FastText(vocab=48844, size=100, alpha=0.025)


In [ ]:
model2 = FastText.load("modelMade.bin")

In [104]:
print(model['java'])

[-0.2669821   0.19755743 -0.0046992   0.26847562 -0.5322416   0.26696637
 -0.17215995  0.07305979 -0.28662375  0.15376262  0.00633639  0.2126481
 -0.73215675  0.05597483 -0.20488891 -0.26239404 -0.08281655  0.30479458
  0.7205819  -0.28240165  0.10745807 -0.24942155  0.00275412 -0.25484732
  0.17827763  0.09927472  0.36553347  0.2605893   0.20897573 -0.71728444
 -0.26912042 -0.15356852  0.01409016  0.09744596  0.56668556 -0.13986348
  0.5304728  -0.14538746 -0.21859369 -0.31353867  0.18406703  0.64211553
  0.21063703 -0.51174295  0.3939129  -0.473652    0.40935156 -0.05909926
  0.2582893  -0.6678205   0.18361033  0.01528603  0.08760922 -0.6850136
 -0.1522014   0.4628623   0.10863721 -0.1617625  -0.39947462 -0.06153871
 -0.24516438  1.0718496   0.5341704  -0.24253592  0.10344952  0.15191782
  0.742359   -0.44402105 -0.13729168  0.754114   -0.08599418 -0.17671071
  0.17393844 -0.5034986   0.83768725  0.10790901 -0.16280273 -0.38103434
 -0.35176802  0.45308474 -0.36042014  0.6631558   0.2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
